In [17]:
"""加载配置"""

import os
import pandas as pd
import docx2txt
import re

keywords = [
    "麦吉丽|雅漾|名创|娇韵诗|雅诗兰黛|欧莱雅",
]
# 指定包含.docx文件的文件夹路径
folder_path = r"C:\Users\chen\Desktop\中转文案"
to_path = r"C:\Users\chen\Desktop\文案合集.xlsx"

In [18]:
"""读取文件表格"""

# 创建一个空的DataFrame来存储文档内容
df = pd.DataFrame(columns=["视频名称", "Text"])

# 遍历文件夹中的所有.docx文件
for filename in os.listdir(folder_path):
    if filename.endswith(".docx"):
        file_path = os.path.join(folder_path, filename)
        # 使用docx2txt库提取文本内容
        text = docx2txt.process(file_path)
        # 将文件名和文本内容添加到DataFrame
        df = pd.concat(
            [df, pd.DataFrame({"视频名称": [filename], "Text": [text]})],
            ignore_index=True,
        )

In [19]:
"""数据清洗及导出"""

# 清洗视频名称
df["视频名称"] = df["视频名称"].str.replace(r"_原文", "", regex=True)


# 处理文案分割
def process_text(text):
    sentences = text.split("。")
    formatted_text = "\n\n".join([f"{sentence}。" for sentence in sentences])
    processed_text = (
        formatted_text
        if len(formatted_text.split("\n")[-1]) > 10
        else "\n".join(formatted_text.split("\n")[:-1])
    )
    return processed_text


df["Text"] = df["Text"].apply(process_text)

# 检查是否合规
df["审查"] = "合规"
pattern = "|".join(keywords)
min_char_count = 60


def check_and_mark(text):
    if len(text) > min_char_count and not re.search(pattern, text):
        return "合规"
    else:
        return "不合规"


df["审查"] = df["Text"].apply(check_and_mark)
df["id"] = df["视频名称"].str.split("_").str[0]
df["user"] = df["视频名称"].str.split("_").str[1]
df.insert(0, "id", df.pop("id"))
df.insert(1, "user", df.pop("user"))

# 导出明细表格
df.to_excel(to_path, index=False)